# **Workspace**

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
! rsync -av --progress /content/drive/"My Drive"/African_Translator/from_github/XLM /content --exclude data/for_test --exclude data/for_the_hypothesis --exclude data/para --exclude data/processed --exclude data/testFASTBPE --exclude dumped --exclude apex --exclude .git --exclude tools/mosesdecoder --exclude tools/fastBPE
#! rm -R /content/XLM

sending incremental file list
XLM/
XLM/.gitignore
          1,577 100%    0.00kB/s    0:00:00 (xfr#1, to-chk=89/91)
XLM/CODE_OF_CONDUCT.md
            244 100%    0.30kB/s    0:00:00 (xfr#2, to-chk=88/91)
XLM/CONTRIBUTING.md
            572 100%    0.00kB/s    0:00:00 (xfr#3, to-chk=87/91)
XLM/Copie de generate-embeddings.ipynb
         17,469 100%   17.39kB/s    0:00:00 (xfr#4, to-chk=86/91)
XLM/LICENSE
         19,336 100%    0.00kB/s    0:00:00 (xfr#5, to-chk=85/91)
XLM/PKM-layer.ipynb
         13,810 100%   13.65kB/s    0:00:00 (xfr#6, to-chk=84/91)
XLM/README.md
         39,290 100%    6.22MB/s    0:00:00 (xfr#7, to-chk=83/91)
XLM/apply_bpe_preprocess.sh
            366 100%    0.33kB/s    0:00:01 (xfr#8, to-chk=82/91)
XLM/generate-embeddings.ipynb
          9,613 100%    0.00kB/s    0:00:00 (xfr#9, to-chk=81/91)
XLM/get-data-glue.sh
          6,143 100%    5.54kB/s    0:00:01 (xfr#10, to-chk=80/91)
XLM/get-data-nmt.sh
         15,009 100%    0.00kB/s    0:00:00 (xfr#11, to-chk=79

In [0]:
% bookmark HOME "/content/XLM" 
%cd -b HOME

(bookmark:HOME) -> /content/XLM
/content/XLM


In [0]:
# Dependencies : fastBPE et Moses (l'installation prend moins de temps que la copie)
% cd tools
! git clone https://github.com/moses-smt/mosesdecoder
! git clone https://github.com/glample/fastBPE && cd fastBPE && g++ -std=c++11 -pthread -O3 fastBPE/main.cc -IfastBPE -o fast

/content/XLM/tools
Cloning into 'mosesdecoder'...
remote: Enumerating objects: 30, done.
remote: Counting objects: 100% (30/30), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 147544 (delta 12), reused 15 (delta 5), pack-reused 147514
Receiving objects: 100% (147544/147544), 129.75 MiB | 12.65 MiB/s, done.
Resolving deltas: 100% (113998/113998), done.
Cloning into 'fastBPE'...
remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 59 (delta 0), reused 1 (delta 0), pack-reused 54
Unpacking objects: 100% (59/59), done.


In [0]:
%cd -b HOME

(bookmark:HOME) -> /content/XLM
/content/XLM


# **Data**

In [0]:
import os
import csv

In [0]:
langues_all = [
            # langues presentes dans l'ancien et le nouveau testament
           'Francais', "Anglais",  "BIBALDA_TA_PELDETTA", 'Bulu',  'Guiziga', "Fulfulde_Adamaoua",  
           "Fulfulde_DC", 'KALATA_KO_SC_Gbaya', 'KALATA_KO_DC_Gbaya', 'Kapsiki_DC', 'Tupurri',
           
           # langues presentes uniquement dans le nouveau testament
          'Bafia', 'Dii', 'Ejagham', 'Ghomala', 'Vute', 'Limbum', 'MKPAMAN_AMVOE_Ewondo', 'Mofa', 
           "Mofu_Gudur", "Ngiemboon", 'Doyayo', "Guidar", 'Peere_Nt&Psalms', 'Samba_Leko', 
           "Du_na_sdik_na_wiini_Alaw"
]

In [0]:
def get_data_from_bible(csv_path, books, output_dir, data_type = "para", langues=[], cell_error = "__Error__"):
    """
    csv_path: folder containing the csvs folder
    books : list of the books of the bibles to be considered (there must exist for each of these books a books.csv file in ../csvs/)
    output: folder in which the lens folders will be created (mono or para)
    data_type: monolingual one or parallel one
    languages: list of languages to be considered (these languages must be included in the list of languages above)
    cell_error: text to be used to mark erroneous text pairs during webscrapping (these pairs are excluded from the data)
    """

    # Si aucune langue n'est spécifiée, on selectionne tout les langues
    if langues == [] :
      langues = langues_all
    
    l = len(langues)
    for i in range(l-1):
      for j in range(i+1, l):
        li = langues[i]
        lj = langues[j]
        print(li + '-' + lj)

        repertoire = output_dir+"/"+data_type
        if not os.path.exists(repertoire):
          os.makedirs(repertoire)

        if data_type == "para":
          repertoire = repertoire +"/"+ li +'-'+ lj
          if not os.path.exists(repertoire):
            os.makedirs(repertoire)
          repertoire = [repertoire + "/" + li + '-' + lj + "." for _ in range(2)]
        elif data_type == "mono":
          repertoire = [repertoire + "/" + li, repertoire +"/"+ lj]
          for rep in repertoire :
            if not os.path.exists(rep):
              os.makedirs(rep)
          repertoire = [r +"/" for r in repertoire]

        with open(repertoire[0] + li + '.txt', 'w') as txtfile1:
          with open(repertoire[1] + lj + '.txt', 'w') as txtfile2:
            
            for fichier in books:
              try :
                with open(csv_path+"/csvs/"+fichier+'csv', 'r') as csvfile:
                  f_csv = csv.reader(csvfile)
                  filenames = []
                  try :
                    fieldnames = next(f_csv)
                  except StopIteration as si :
                    print(si)
                  try :
                    index_i = fieldnames.index(li)
                    index_j = fieldnames.index(lj)
                    versert = fieldnames.index("livre.chapitre.verset")
                    for ligne in f_csv:
                      x_i = ligne[index_i] 
                      y_i =  ligne[index_j]
                      if x_i != cell_error and y_i != cell_error :
                          txtfile1.writelines(x_i+'\n')
                          txtfile2.writelines(y_i+'\n')
                  except Exception as ex:
                    print(ex)
              except Exception as ex :
                print(ex)
                pass

In [0]:
livres = [
    # Ancien testatement
    'GEN.', 'EXO.', 'LEV.', 'NUM.', 'DEU.', 'JOS.', 'JDG.', 'RUT.',   '1SA.', '2SA.', '1KI.', '2KI.', 
    '1CH.', '2CH.', 'EZR.', 'NEH.', 'EST.', 'JOB.', 'PSA.', 'PRO.', 'ECC.', 'SNG.',  'ISA.', 'JER.', 
    'LAM.', 'EZK.', 'DAN.', 'HOS.', 'JOL.', 'AMO.', 'OBA.', 'JON.', 'MIC.', 'NAM.', 'HAB.', 'ZEP.', 
    'HAG.', 'ZEC.', 'MAL.',

    # Nouveau testament
    'MAT.', 'MRK.', 'LUK.', 'JHN.', 'ACT.', 'ROM.', '1CO.', '2CO.', 'GAL.', 'EPH.', 'PHP.', 'COL.', 
    '1TH.', '2TH.', '1TI.', '2TI.', 'TIT.', 'PHM.', 'HEB.', 'JAS.', '1PE.',  '2PE.', '1JN.', '2JN.', 
    '3JN.', 'JUD.', 'REV.'
]

In [0]:
csv_path = "/content/drive/My Drive/African_Translator/datasets/YourVersion"
output_dir = "/content/XLM/data/africa"
! mkdir /content/XLM/data/africa
data_type = "para"
langues = ["Anglais", "Bulu"]

mkdir: cannot create directory ‘/content/XLM/data/africa’: File exists


In [0]:
get_data_from_bible(csv_path = csv_path, livres = livres, output_dir = output_dir, data_type = data_type, langues = langues)

Anglais-Bulu


In [0]:
# dossier (contenant/dans le quel stocker) les données 
% env PARA_PATH=data/africa/para
! mkdir -p $PARA_PATH

% env nCodes=30000
% env shuf_n_samples=100000

# Pourcentage des données de test et de validation (en %) 
# todo : bon ? 
% env test_size=10
% env val_size=10

env: PARA_PATH=data/africa/para
env: nCodes=30000
env: shuf_n_samples=100000
env: test_size=10
env: val_size=10


In [0]:
# tools paths
#% env TOKENIZE=$TOOLS_PATH/tokenize.sh
% env TOKENIZE=tools/tokenizer_our.sh
#% env LOWER_REMOVE_ACCENT=$TOOLS_PATH/lowercase_and_remove_accent.py
% env LOWER_REMOVE_ACCENT=tools/lowercase_and_remove_accent.py
#% env FASTBPE=$TOOLS_PATH/fastBPE/fast
% env FASTBPE=tools/fastBPE/fast

#% env OUTPATH $PROCESSED_PATH/$pair/$nCodes
# nommage du fichier en fonction de 'nCodes'
% env OUTPATH data/africa/processed/30000

! mkdir -p $OUTPATH

! chmod +x $FASTBPE
! chmod +x get_data_hypothesis.sh
! chmod +x tools/mosesdecoder/scripts/tokenizer/*.perl

env: TOKENIZE=tools/tokenizer_our.sh
env: LOWER_REMOVE_ACCENT=tools/lowercase_and_remove_accent.py
env: FASTBPE=tools/fastBPE/fast
env: OUTPATH=data/africa/processed/30000


In [0]:
% env threads_for_tokenizer=16

env: threads_for_tokenizer=16


In [0]:
! chmod +x get_africa_data.sh
! ./get_africa_data.sh Anglais-Bulu 

dir data/africa/para/Anglais-Bulu already exists


*** Cleaning and tokenizing Anglais-Bulu data ... ***
Tokenizer Version 1.1
Language: Anglais
Number of threads: 16
*** Tokenized (+ lowercase + accent-removal) Anglais-Bulu.Anglais data to data/africa/para/Anglais-Bulu/? ***
Tokenizer Version 1.1
Language: Bulu
Number of threads: 16
*** Tokenized (+ lowercase + accent-removal) Anglais-Bulu.Bulu data to data/africa/para/Anglais-Bulu/? ***


*** split into train / valid / test ***



***build the training set for BPE tokenization (30000 codes)***


***shuf ... Generating 100000 random permutations of train data and store result in data/africa/processed/30000/Anglais-Bulu/bpe.train***


***Learn the BPE vocabulary on the training set : data/africa/processed/30000/bpe.train***
Loading vocabulary from data/africa/processed/30000/Anglais-Bulu/bpe.train ...
Read 6038010 words (22141 unique) from text file.
tcmalloc: large alloc 12000002048 bytes == 0x562ec9a70000 @  0x7f5059cb7887 0x562ec874

In [0]:
#! rm -R /content/XLM/data/africa/para
#! rm -R /content/XLM/data/africa/processed

# **Dépendances**

In [0]:
## apex
%cd /content
! git clone https://github.com/NVIDIA/apex
%cd /content/apex
! pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./

/content
Cloning into 'apex'...
remote: Enumerating objects: 6742, done.
remote: Total 6742 (delta 0), reused 0 (delta 0), pack-reused 6742
Receiving objects: 100% (6742/6742), 13.74 MiB | 3.22 MiB/s, done.
Resolving deltas: 100% (4503/4503), done.
/content/apex
/usr/local/lib/python3.6/dist-packages/pip/_internal/commands/install.py:283: UserWarning: Disabling all use of wheels due to the use of --build-options / --global-options / --install-options.
  cmdoptions.check_install_build_global(options)
Created temporary directory: /tmp/pip-ephem-wheel-cache-9qfistk2
Created temporary directory: /tmp/pip-req-tracker-pou2zn5e
Created requirements tracker '/tmp/pip-req-tracker-pou2zn5e'
Created temporary directory: /tmp/pip-install-zz0fghxd
Processing /content/apex
  Created temporary directory: /tmp/pip-req-build-fhafgr5q
  Added file:///content/apex to build tracker '/tmp/pip-req-tracker-pou2zn5e'
    Running setup.py (path:/tmp/pip-req-build-fhafgr5q/setup.py) egg_info for package from fi

# **Train**

In [0]:
%env exp_id=africa
#%env batch_size=32
%env batch_size=16
%env lgs=es-fr
# Maximum length of sentences (after BPE)
%env max_len=100

env: exp_id=africa
env: batch_size=16
env: lgs=es-fr
env: max_len=100


In [0]:
# le plus grand fichier à tensor_len phrases, on cherche le multiple de "batch_size" le plus proche de ce 
# nombre par valeur supérieur : epoch_size doit etre un multiple non nul de ce nombre (pour ne pas gaspiller) 

main_path = "/content/XLM/data/africa/para/"
        
import io

def n_lines(file_path):
    return len(io.open(file_path, encoding='UTF-8').read().split('\n'))

tensor_len = n_lines(file_path = main_path + "Anglais-Bulu/Anglais-Bulu.Anglais.train")
print("tensor_len = " + str(tensor_len))

tensor_len = 22855


In [0]:
tensor_len=22855
#batch_size=32
batch_size=16

def getEpochSize(tensor_len, batch_size):
    i = tensor_len
    while True :
        if i%batch_size == 0 :
            return i//batch_size
        i = i + 1
        
epoch_size = getEpochSize(tensor_len, batch_size)
print("epoch_size = " + str(epoch_size))

epoch_size = 1429


In [0]:
# Et deduire epoch_size (evaluation frequency, -1 for parallel data size)
%env epoch_size=1429
#%env epoch_size=-1

env: epoch_size=1429


In [0]:
%env remove_long_sentences_train=True
%env remove_long_sentences_valid=True
%env remove_long_sentences_test=True
#--remove_long_sentences_train $remove_long_sentences_train --remove_long_sentences_valid $remove_long_sentences_valid --remove_long_sentences_test $remove_long_sentences_test

env: remove_long_sentences_train=True
env: remove_long_sentences_valid=True
env: remove_long_sentences_test=True


In [0]:
# limit the number of examples (-1 by default for non limitation)
%env train_n_samples=-1
%env valid_n_samples=-1
%env test_n_samples=-1
#--train_n_samples $train_n_samples --valid_n_samples $valid_n_samples --test_n_samples $test_n_samples

env: train_n_samples=-1
env: valid_n_samples=-1
env: test_n_samples=-1


**Pretrain a language model**

In [0]:
# stopping criterion (if criterion does not improve 10 times)
%env stopping_criterion=_valid_mlm_ppl,10
%env eval_bleu false
%env lgs=Anglais-Bulu
%env mlm_steps=Anglais,Bulu,Anglais-Bulu

env: stopping_criterion=_valid_mlm_ppl,10
env: eval_bleu=false
env: lgs=Anglais-Bulu
env: mlm_steps=Anglais,Bulu,Anglais-Bulu


In [0]:
%env max_epoch=18
#! export NGPU=1; python -m torch.distributed.launch --nproc_per_node=$NGPU train.py --exp_name esit_mlm_tlm --dump_path ./dumped/ --data_path $OUTPATH --lgs $lgs --clm_steps '' --mlm_steps $mlm_steps --emb_dim 1024 --n_layers 6 --n_heads 8 --dropout 0.1 --attention_dropout 0.1 --gelu_activation true --batch_size $batch_size --bptt 256 --optimizer adam,lr=0.0001 --epoch_size $epoch_size --max_epoch $max_epoch --validation_metrics _valid_mlm_ppl --stopping_criterion $stopping_criterion --eval_bleu $eval_bleu --exp_id $exp_id --remove_long_sentences_train $remove_long_sentences_train --remove_long_sentences_valid $remove_long_sentences_valid --remove_long_sentences_test $remove_long_sentences_test --train_n_samples $train_n_samples --valid_n_samples $valid_n_samples --test_n_samples $test_n_samples 
! python train.py --exp_name Anglais_Bulu_mlm_tlm --dump_path ./dumped/ --data_path $OUTPATH/Anglais-Bulu --lgs $lgs --clm_steps '' --mlm_steps $mlm_steps --emb_dim 1024 --n_layers 6 --n_heads 8 --dropout 0.1 --attention_dropout 0.1 --gelu_activation true --batch_size $batch_size --bptt 256 --optimizer adam,lr=0.0001 --epoch_size $epoch_size --max_epoch $max_epoch --validation_metrics _valid_mlm_ppl --stopping_criterion $stopping_criterion --eval_bleu $eval_bleu --exp_id $exp_id --remove_long_sentences_train $remove_long_sentences_train --remove_long_sentences_valid $remove_long_sentences_valid --remove_long_sentences_test $remove_long_sentences_test --train_n_samples $train_n_samples --valid_n_samples $valid_n_samples --test_n_samples $test_n_samples 

env: max_epoch=18
FAISS library was not found.
FAISS not available. Switching to standard nearest neighbors search implementation.
SLURM job: False
0 - Number of nodes: 1
0 - Node ID        : 0
0 - Local rank     : 0
0 - Global rank    : 0
0 - World size     : 1
0 - GPUs per node  : 1
0 - Master         : True
0 - Multi-node     : False
0 - Multi-GPU      : False
0 - Hostname       : 09297f1b806a
INFO - 05/09/20 13:10:26 - 0:00:00 - ============ Initialized logger ============
INFO - 05/09/20 13:10:26 - 0:00:00 - accumulate_gradients: 1
                                     ae_steps: []
                                     amp: -1
                                     asm: False
                                     attention_dropout: 0.1
                                     batch_size: 16
                                     beam_size: 1
                                     bptt: 256
                                     bt_src_langs: []
                                     bt_steps: []
 

In [0]:
#! rm -R dumped/Anglais_Bulu_mlm_tlm

**Train on unsupervised MT from a pretrained model**

In [0]:
#%env batch_size=...
#%env epoch_size=...

%env eval_bleu=true
# comme eval_bleu=true
%cd /content/XLM
! chmod +x src/evaluation/multi-bleu.perl

env: eval_bleu=true
/content/XLM


In [0]:
#%env stopping_criterion=valid_de-fr_mt_bleu,10
%env stopping_criterion=valid_Anglais-Bulu_mt_bleu,5
%env validation_metrics=valid_Anglais-Bulu_mt_bleu
%env reload_model=dumped/Anglais_Bulu_mlm_tlm/africa/best-valid_mlm_ppl.pth,dumped/Anglais_Bulu_mlm_tlm/africa/best-valid_mlm_ppl.pth
%env ae_steps=Anglais,Bulu
%env bt_steps=Anglais-Bulu-Anglais,Bulu-Anglais-Bulu

env: stopping_criterion=valid_Anglais-Bulu_mt_bleu,5
env: validation_metrics=valid_Anglais-Bulu_mt_bleu
env: reload_model=dumped/Anglais_Bulu_mlm_tlm/africa/best-valid_mlm_ppl.pth,dumped/Anglais_Bulu_mlm_tlm/africa/best-valid_mlm_ppl.pth
env: ae_steps=Anglais,Bulu
env: bt_steps=Anglais-Bulu-Anglais,Bulu-Anglais-Bulu


In [0]:
%env max_epoch=2
%env batch_size=16
%env epoch_size=22000
! python train.py --exp_name unsupMT_Anglais_Bulu --dump_path ./dumped/ --reload_model $reload_model --data_path $OUTPATH/Anglais-Bulu --lgs $lgs --ae_steps $ae_steps --bt_steps $bt_steps --word_shuffle 3 --word_dropout 0.1 --word_blank 0.1 --lambda_ae '0:1,100000:0.1,300000:0' --encoder_only false --emb_dim 1024 --n_layers 6 --n_heads 8 --dropout 0.1 --attention_dropout 0.1 --gelu_activation true --tokens_per_batch 2000 --batch_size $batch_size --bptt 256 --optimizer adam_inverse_sqrt,beta1=0.9,beta2=0.98,lr=0.0001 --epoch_size $epoch_size --max_epoch $max_epoch --eval_bleu $eval_bleu --stopping_criterion $stopping_criterion --validation_metrics $validation_metrics --exp_id $exp_id        
#%env mt_steps=Anglais-Bulu,Bulu-Anglais           
#! python train.py --exp_name unsupMT_Anglais_Bulu --dump_path ./dumped/ --reload_model $reload_model --data_path $OUTPATH/Anglais-Bulu --lgs $lgs --ae_steps $ae_steps --mt_steps $mt_steps --bt_steps $bt_steps --word_shuffle 3 --word_dropout 0.1 --word_blank 0.1 --lambda_ae '0:1,100000:0.1,300000:0' --encoder_only false --emb_dim 1024 --n_layers 6 --n_heads 8 --dropout 0.1 --attention_dropout 0.1 --gelu_activation true --tokens_per_batch 2000 --batch_size $batch_size --bptt 256 --optimizer adam_inverse_sqrt,beta1=0.9,beta2=0.98,lr=0.0001 --epoch_size $epoch_size --max_epoch $max_epoch --eval_bleu $eval_bleu --stopping_criterion $stopping_criterion --validation_metrics $validation_metrics --exp_id $exp_id        

env: max_epoch=2
env: batch_size=16
env: epoch_size=22000
FAISS library was not found.
FAISS not available. Switching to standard nearest neighbors search implementation.
SLURM job: False
0 - Number of nodes: 1
0 - Node ID        : 0
0 - Local rank     : 0
0 - Global rank    : 0
0 - World size     : 1
0 - GPUs per node  : 1
0 - Master         : True
0 - Multi-node     : False
0 - Multi-GPU      : False
0 - Hostname       : 09297f1b806a
INFO - 05/09/20 13:47:06 - 0:00:00 - ============ Initialized logger ============
INFO - 05/09/20 13:47:06 - 0:00:00 - accumulate_gradients: 1
                                     ae_steps: ['Anglais', 'Bulu']
                                     amp: -1
                                     asm: False
                                     attention_dropout: 0.1
                                     batch_size: 16
                                     beam_size: 1
                                     bptt: 256
                                     bt_src_lang

In [0]:
#! rm -R dumped/unsupMT_Anglais_Bulu